In [1]:
cd ..

C:\Users\agus\Dropbox\Doctorado\NeuroData\PyLeech


In [2]:
cd ..

C:\Users\agus\Dropbox\Doctorado\NeuroData


In [3]:
%matplotlib qt5

In [4]:
import PyLeech.Utils.CrawlingDatabaseUtils as CDU
import PyLeech.Utils.burstClasses as burstClasses
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import PyLeech.Utils.AbfExtension as abfe
from PyLeech.Utils.burstStorerLoader import BurstStorerLoader
import PyLeech.Utils.burstUtils as burstUtils
import numpy as np
import PyLeech.Utils.NLDUtils as NLD
import scipy.signal as spsig
import matplotlib as mpl
import os
from matplotlib import animation
import matplotlib
plt.rcParams['animation.ffmpeg_path'] = "C:\\ffmpeg\\bin\\ffmpeg.exe"
font = {'family' : 'normal',
        'weight' : 'normal',
        'size'   : 28}
matplotlib.rcParams['figure.figsize'] = (20, 10)
matplotlib.rc('font', **font)

In [5]:
cdd = CDU.loadDataDict("RegistrosDP_PP/embeddings_CDD.json")
file_list = list(cdd)
k = 1
draw_everything = False
draw_segments_by_file = True
mean_tray_dict = {}


segments_by_file = {file_list[0]: [[1, 4], [2, 7, 9], [15, 18, 19, 20, 22]],
                   file_list[1]: [[0, 4, 6, 8, 18, 25], [5, 7, 11, 13, 16], [20, 21, 26, 28, 29]],
                   file_list[2]: [[1, 4, 5, 6, 7, 10], [9, 12], [11, 16, 19, 21, 22, 27, 30, 33, 35, 36, 38, 42], [26, 28, 44], [31, 37, 39, 41]],
                   file_list[3]: [[0, 1, 4, 5], [2, 3, 5, 6, 7, 8, 10, 11], [19, 25], [17, 20, 21], [26, 27, 28]],
                   file_list[4]: [[0, 1, 2, 4, 5, 6, 7, 8], [3, 9], [17, 20, 21], [14, 19, 24, 25], [16, 18, 22, 26, 27, 28]],
                   file_list[5]: [[1, 4], [8, 10], [5, 7, 9], [0, 2, 3]]
                  }

In [16]:
    for fn in [file_list[2]]:
        # if cdd[fn]["skipped"]:
        #     continue
        # print(fn)
        try:
            burst_object = BurstStorerLoader(fn, 'RegistrosDP_PP', 'load')
            try:
                arr_dict, time_vector, fs = abfe.getArraysFromAbfFiles(fn, ["IN5"])
            except:
                pass
        except:
            print("final expeption")
            print(Exception)
            continue

        good_neurons = [neuron for neuron, neuron_dict in cdd[fn]['neurons'].items() if neuron_dict['neuron_is_good']]

        crawling_intervals = cdd[fn]['crawling_intervals']
        print(crawling_intervals)
        binning_dt = 0.1
        spike_kernel_sigma = 1
        NS_kernel_sigma = 1

        spike_freq_array = burstUtils.processSpikeFreqDict(burst_object.spike_freq_dict, .1, selected_neurons=good_neurons,
                                                           time_length=burst_object.time[-1])

        kernel = NLD.generateGaussianKernel(sigma=spike_kernel_sigma, time_range=20, dt_step=binning_dt)
        smoothed_sfd = {}
        for key, items in spike_freq_array.items():
            smoothed_sfd[key] = np.array([items[0], spsig.fftconvolve(items[1], kernel, mode='same')])

        step = int(binning_dt * fs)
        neuron_idxs = []
        spike_idxs = []
        analized_neuron = list(smoothed_sfd)[0]
        for interval in crawling_intervals:
            neuron_idxs.append(np.where((burst_object.time > interval[0]) & (burst_object.time < interval[1]))[0][::step])
            spike_idxs.append(np.where(
                (smoothed_sfd[analized_neuron][0] > interval[0]) & (smoothed_sfd[analized_neuron][0] < interval[1]))[0])
        neuron_idxs = np.hstack(neuron_idxs)
        spike_idxs = np.hstack(spike_idxs)
        neuron_cut_time = burst_object.time[neuron_idxs]

        fig, axes = burstUtils.plotFreq(burst_object.spike_freq_dict, color_dict=burst_object.color_dict,
                                        template_dict=burst_object.template_dict, scatter_plot=True,
                                        outlier_thres=3, ms=4, draw_list=good_neurons)

        fig.suptitle(fn)
        sp_embedding = NLD.getDerivativeEmbedding(smoothed_sfd[analized_neuron][1][spike_idxs], .1, 3)
        freq_threshold = 1
        if fn == file_list[2]:
            num = 50
            freq_threshold = 1
        elif fn == file_list[0]:
            num = 250
            freq_threshold = 1
        elif fn == "RegistrosDP_PP\\2018_11_06_0004.pklspikes":
            print("TRACE IS NOT FROM NOT DE3")
            analized_neuron = 22
        else:
            num = 250

        sp_embedding[(sp_embedding[:, 2] > num) | (sp_embedding[:, 2] < -num)] = np.nan

        idxs = burstUtils.getNonZeroIdxs(sp_embedding[:, 0], freq_threshold)
        intervals = idxs.reshape(int(idxs.shape[0] / 2), 2)

        fig, ax = plt.subplots()
        ax.plot(smoothed_sfd[analized_neuron][0][spike_idxs], sp_embedding[:, 0])
        j = 0
        for i in intervals.flatten():
            ax.axvline(smoothed_sfd[analized_neuron][0][spike_idxs][i], c='r')
            if j % 2 == 0:
                ax.text(smoothed_sfd[analized_neuron][0][spike_idxs][i], 0, str(int(j / 2)))
            j += 1
        fig.suptitle(fn)
        step = 5

        if draw_everything:
            j = 0
            for i in np.arange(0, intervals.shape[0], step):

                fig = plt.figure()
                ax = Axes3D(fig)

                cmap = plt.get_cmap('jet')
                cNorm = mpl.colors.Normalize(vmin=0, vmax=step - 1)
                scalarMap = mpl.cm.ScalarMappable(norm=cNorm, cmap=cmap)

                data_mean = NLD.getSegmentsMean(sp_embedding, intervals[i:i + step])
                iter = 0
                max_int = np.max(np.diff(intervals[i:i + step]))

                for start, end in intervals[i:i + step]:
                    diff = end - start
                    NLD.plot3Dline(sp_embedding[start:end] * diff / max_int, fig_ax_pair=[fig, ax], colorbar=False,
                                   color=scalarMap.to_rgba(iter), label=str(j))
                    iter += 1
                    j += 1

                NLD.plot3Dline(data_mean, fig_ax_pair=[fig, ax], colorbar=False, color=None)

                fig.suptitle(os.path.basename(fn) + '\n' + str(i) + ':' + str(i + step))
                fig.legend()
            plt.show()
            plt.ginput(n=0, timeout=0, mouse_pop=2, mouse_stop=3)
            plt.close('all')

        try:
            segments = segments_by_file[fn]
        except KeyError:
            print("No segment for file %s" % fn)
            segments = []
        
        figuras = []
        if draw_segments_by_file:
            fig1 = plt.figure()
            ax1 = Axes3D(fig1)
            j = 0
            means_list = []
            for interval_list in segments:
                data_mean = NLD.getSegmentsMean(sp_embedding, intervals[interval_list])

                means_list.append(data_mean)

                cmap = plt.get_cmap('jet')
                cNorm = mpl.colors.Normalize(vmin=0, vmax=len(interval_list) - 1)
                scalarMap = mpl.cm.ScalarMappable(norm=cNorm, cmap=cmap)

                fig = plt.figure(figsize=(20,15))
                ax = Axes3D(fig)
                iter = 0
                max_int = np.max(np.diff(intervals[interval_list]))
                for elem in interval_list:
                    diff = np.diff(intervals[elem])
                    NLD.plot3Dline(sp_embedding[intervals[elem, 0]:intervals[elem, 1]] * diff / max_int,
                                   fig_ax_pair=[fig, ax],
                                   color=scalarMap.to_rgba(iter),
                                   colorbar=False, label=str(elem))
                    iter += 1
#                 fig.suptitle(str(interval_list))
                NLD.plot3Dline(data_mean, fig_ax_pair=[fig, ax], colorbar=False, color='k')


                NLD.plot3Dline(data_mean, fig_ax_pair=[fig1, ax1], colorbar=False, color='k')
                figuras.append([str(interval_list), fig])
                j += 1
                fig.tight_layout()
                fig.savefig("figuras_charla/" + str(interval_list) + ".png", dpi=600, transparent=True)
                ax.set_frame_on(False)
                ax.grid(b=False)
#                 ax.set_frame_on(False)
                ax.set_facecolor('k')
                fig.set_facecolor('k')
                ax.w_xaxis.pane.fill = False
                ax.w_yaxis.pane.fill = False
                ax.w_zaxis.pane.fill = False
                ax._axis3don = False
                def animate(i):
                    ax.view_init(azim=i)
                    return fig,
                anim = animation.FuncAnimation(fig, animate, frames=np.arange(0,360.5,0.5),
                                               interval=100, blit=True)
                mywriter = animation.FFMpegWriter(fps=30, extra_args=['-vcodec', 'libx264'])
                anim.save("figuras_charla/" + str(interval_list) + "_black.mp4",  writer=mywriter)

                # fig.legend()

            plt.show()
            # plt.ginput(n=0, timeout=0, mouse_pop=2, mouse_stop=3)
            # plt.close('all')
            mean_tray_dict[fn] = means_list
            fig1.savefig("figuras_charla/" + str(interval_list) + "_black.png", dpi=600, transparent=True)
        else:
            means_list = []
            for interval_list in segments:
                data_mean = NLD.getSegmentsMean(sp_embedding, intervals[interval_list])

                means_list.append(data_mean)
            mean_tray_dict[fn] = means_list


[[350, 623], [662, 960]]


c:\users\agus\appdata\local\programs\python\python36\lib\site-packages\findiff\findiff.py:242: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  yd[ref_multi_slice] += w * y[off_multi_slice]
c:\users\agus\appdata\local\programs\python\python36\lib\site-packages\findiff\findiff.py:240: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  yd[ref_multi_slice] += y[off_multi_slice]
C:\Users\agus\Dropbox\Doctorado\NeuroData\PyLeech\Utils\burstUtils.py:103: RuntimeWarning: invalid value encountered in greater
  non_zero_idxs = np.where(trace > threshold)[

In [15]:
ax.set_facecolor('black')
ax.w_xaxis.pane.fill = False
ax.w_yaxis.pane.fill = False
ax.w_zaxis.pane.fill = False
ax._axis3don = False

In [ ]:
fig = plt.figure()
    ax = Axes3D(fig)
    cmap = plt.get_cmap('jet')
    cNorm = mpl.colors.Normalize(vmin=0, vmax=len(list(mean_tray_dict)) - 1)
    scalarMap = mpl.cm.ScalarMappable(norm=cNorm, cmap=cmap)

    for key, items in mean_tray_dict.items():
        first = True
        j = 0
        cNorm = mpl.colors.Normalize(vmin=0, vmax=len(items) - 1)
        scalarMap = mpl.cm.ScalarMappable(norm=cNorm, cmap=cmap)
        for orbit in items:
            if first:
                NLD.plot3Dline(orbit, fig_ax_pair=[fig, ax], colorbar=False, color=scalarMap.to_rgba(j),
                               label=os.path.basename(os.path.splitext(key)[0]))
                first = False
            else:
                NLD.plot3Dline(orbit, fig_ax_pair=[fig, ax], colorbar=False, color=scalarMap.to_rgba(j))
            j += 1
    # fig.legend()
    # fig.savefig("figuras_charla/every_embedding.png", dpi=600, transparent=True)
    plt.show()

In [ ]:
fig = plt.figure(figsize=(20,15))
ax = Axes3D(fig)
NLD.plot3Dline(sp_embedding[:int(3*sp_embedding.shape[0]/4)], fig_ax_pair=[fig, ax], colorbar=True, linewidth=1.5)
def animate(i):
    ax.view_init(azim=i)
    return fig,
anim = animation.FuncAnimation(fig, animate, frames=np.arange(0,360.5,0.5),
                               interval=100, blit=True)
mywriter = animation.FFMpegWriter(fps=30, extra_args=['-vcodec', 'libx264'])
anim.save("figuras_charla/full_emb.mp4",  writer=mywriter)